In [1]:
import numpy as np
import tensorflow as tf
import gc

2023-02-11 11:24:08.721343: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from scipy.cluster.vq import kmeans
import scipy
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.layers import Input, Add, Multiply, Lambda, Concatenate
from tensorflow.keras.models import Model
import numpy as np
import tensorflow as tf
tf.keras.backend.set_floatx('float64')

class AbstractDictionary(object):
    def generate_B(self, inputs):
        target_dim = inputs.shape[-1]
        self.basis_func_number = self.n_dic_customized + target_dim + 1
        # Form B matrix
        self.B = np.zeros((self.basis_func_number, target_dim))
        for i in range(0, target_dim):
            self.B[i + 1][i] = 1
        return self.B


class DicNN(Layer):
    """Trainable dictionaries
    """

    def __init__(self, layer_sizes=[64, 64], n_psi_train=22, **kwargs):
        super(DicNN, self).__init__(**kwargs)
        self.layer_sizes = layer_sizes
        self.input_layer = Dense(self.layer_sizes[0], use_bias=False)
        self.hidden_layers = [Dense(s, activation='tanh') for s in layer_sizes]
        self.output_layer = Dense(n_psi_train)
        self.n_psi_train = n_psi_train

    def call(self, inputs):
        psi_x_train = self.input_layer(inputs)
        for layer in self.hidden_layers:
            psi_x_train = psi_x_train + layer(psi_x_train)
        outputs = self.output_layer(psi_x_train)
        return outputs

    def get_config(self):
        config = super(DicNN, self).get_config()
        config.update({
            'layer_sizes': self.layer_sizes,
            'n_psi_train': self.n_psi_train
        })
        return config


class PsiNN(Layer, AbstractDictionary):
    """Concatenate constant, data and trainable dictionaries together as [1, data, DicNN]
    """

    def __init__(
            self,
            dic_trainable=DicNN,
            layer_sizes=[
                64,
                64],
            n_psi_train=22,
            **kwargs):
        super(PsiNN, self).__init__(**kwargs)
        self.layer_sizes = layer_sizes
        self.dic_trainable = dic_trainable
        self.n_dic_customized = n_psi_train
        self.dicNN = self.dic_trainable(
            layer_sizes=self.layer_sizes,
            n_psi_train=self.n_dic_customized)

    def call(self, inputs):
        constant = tf.ones_like(tf.slice(inputs, [0, 0], [-1, 1]))
        psi_x_train = self.dicNN(inputs)
        outputs = Concatenate()([constant, inputs, psi_x_train])
        return outputs

    def get_config(self):
        config = super(PsiNN, self).get_config()
        config.update({
            'dic_trainable': self.dic_trainable,
            'layer_sizes': self.layer_sizes,
            'n_psi_train': self.n_dic_customized
        })
        return config


In [3]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# load MNIST data.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# Preprocessing.
x_train = x_train / 255.0
x_test = x_test / 255.0
shuffle_index = np.random.permutation(len(x_train))
x_train, y_train = x_train[shuffle_index], y_train[shuffle_index]
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
# Track the data type.
dataType = x_train.dtype
print(f"Data type: {dataType}")
labelType = y_test.dtype
print(f"Data type: {labelType}")

In [ ]:
im_list = []
n_samples_to_show = 16
c = 0
for i in range(n_samples_to_show):
  im_list.append(x_train[i])
# Visualization
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
fig = plt.figure(figsize=(4., 4.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(4, 4),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )
# Show image grid
for ax, im in zip(grid, im_list):
    # Iterating over the grid returns the Axes.
    ax.imshow(im, 'gray')
plt.show()

In [ ]:
# Instantiate the training dataset.
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))
print(x_train.shape)

In [96]:
# PCA
from sklearn.preprocessing import StandardScaler
x_std = StandardScaler().fit_transform(x_train)
x_std_test = StandardScaler().fit_transform(x_test)

# Call the PCA method with 5 components. 
pca = PCA(n_components=5)
pca.fit(x_std)
x_5d = pca.transform(x_std)
pca.fit(x_std_test)
x_5d_test = pca.transform(x_std_test)

print(x_5d.shape)

del x_std, x_train, x_test, x_std_test
gc.collect()

# ref:https://blog.csdn.net/capecape/article/details/79163454

(60000, 5)


70143

In [102]:
# Create the model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

model_ref = Sequential()
model_ref.add(Dense(units=64, activation='relu'))
# model_ref.add(Dense(units=16, activation='relu'))
model_ref.add(Dense(units=10, activation='softmax'))

In [105]:
from tensorflow import keras

# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=0.001)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
epochs = 100

In [52]:
for epoch in range(epochs):
#     print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    with tf.GradientTape() as tape:
        logits = model_ref(x_5d, training = True)
        loss_value = loss_fn(y_train, logits)
    grads = tape.gradient(loss_value, model_ref.trainable_weights)
    optimizer.apply_gradients(zip(grads, model_ref.trainable_weights))
    print("Training loss at epoch %d: %.4f" % (epoch, float(loss_value)))

Training loss at epoch 0: 1.1354
Training loss at epoch 1: 1.1353
Training loss at epoch 2: 1.1352
Training loss at epoch 3: 1.1351
Training loss at epoch 4: 1.1350
Training loss at epoch 5: 1.1349
Training loss at epoch 6: 1.1348
Training loss at epoch 7: 1.1347
Training loss at epoch 8: 1.1346
Training loss at epoch 9: 1.1345
Training loss at epoch 10: 1.1344
Training loss at epoch 11: 1.1343
Training loss at epoch 12: 1.1342
Training loss at epoch 13: 1.1341
Training loss at epoch 14: 1.1340
Training loss at epoch 15: 1.1339
Training loss at epoch 16: 1.1339
Training loss at epoch 17: 1.1338
Training loss at epoch 18: 1.1337
Training loss at epoch 19: 1.1336
Training loss at epoch 20: 1.1335
Training loss at epoch 21: 1.1334
Training loss at epoch 22: 1.1333
Training loss at epoch 23: 1.1332
Training loss at epoch 24: 1.1331
Training loss at epoch 25: 1.1330
Training loss at epoch 26: 1.1329
Training loss at epoch 27: 1.1328
Training loss at epoch 28: 1.1327
Training loss at epoch 2

In [106]:
model_ref.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model_ref.fit(x=x_5d,y=y_train,epochs=1000,batch_size=x_5d.shape[0],validation_data=(x_5d_test, y_test))

Epoch 1/1000
1/1 [==============================] - 1s 545ms/step - loss: 0.9979 - accuracy: 0.6585 - val_loss: 1.2764 - val_accuracy: 0.5740
Epoch 2/1000
1/1 [==============================] - 0s 113ms/step - loss: 0.9978 - accuracy: 0.6586 - val_loss: 1.2763 - val_accuracy: 0.5740
Epoch 3/1000
1/1 [==============================] - 0s 141ms/step - loss: 0.9977 - accuracy: 0.6585 - val_loss: 1.2762 - val_accuracy: 0.5741
Epoch 4/1000
1/1 [==============================] - 0s 131ms/step - loss: 0.9976 - accuracy: 0.6585 - val_loss: 1.2761 - val_accuracy: 0.5741
Epoch 5/1000
1/1 [==============================] - 0s 166ms/step - loss: 0.9975 - accuracy: 0.6586 - val_loss: 1.2761 - val_accuracy: 0.5742
Epoch 6/1000
1/1 [==============================] - 0s 166ms/step - loss: 0.9974 - accuracy: 0.6586 - val_loss: 1.2760 - val_accuracy: 0.5741
Epoch 7/1000
1/1 [==============================] - 0s 143ms/step - loss: 0.9973 - accuracy: 0.6587 - val_loss: 1.2759 - val_accuracy: 0.5741
Epoch 

1/1 [==============================] - 0s 116ms/step - loss: 0.9874 - accuracy: 0.6613 - val_loss: 1.2674 - val_accuracy: 0.5786
Epoch 116/1000
1/1 [==============================] - 0s 126ms/step - loss: 0.9873 - accuracy: 0.6613 - val_loss: 1.2674 - val_accuracy: 0.5787
Epoch 117/1000
1/1 [==============================] - 0s 174ms/step - loss: 0.9873 - accuracy: 0.6613 - val_loss: 1.2673 - val_accuracy: 0.5789
Epoch 118/1000
1/1 [==============================] - 0s 131ms/step - loss: 0.9872 - accuracy: 0.6612 - val_loss: 1.2672 - val_accuracy: 0.5789
Epoch 119/1000
1/1 [==============================] - 0s 110ms/step - loss: 0.9871 - accuracy: 0.6612 - val_loss: 1.2672 - val_accuracy: 0.5789
Epoch 120/1000
1/1 [==============================] - 0s 156ms/step - loss: 0.9870 - accuracy: 0.6613 - val_loss: 1.2671 - val_accuracy: 0.5789
Epoch 121/1000
1/1 [==============================] - 0s 123ms/step - loss: 0.9869 - accuracy: 0.6612 - val_loss: 1.2670 - val_accuracy: 0.5788
Epoch 1

Epoch 172/1000
1/1 [==============================] - 0s 129ms/step - loss: 0.9826 - accuracy: 0.6624 - val_loss: 1.2633 - val_accuracy: 0.5794
Epoch 173/1000
1/1 [==============================] - 0s 180ms/step - loss: 0.9825 - accuracy: 0.6623 - val_loss: 1.2633 - val_accuracy: 0.5794
Epoch 174/1000
1/1 [==============================] - 0s 177ms/step - loss: 0.9824 - accuracy: 0.6624 - val_loss: 1.2632 - val_accuracy: 0.5794
Epoch 175/1000
1/1 [==============================] - 0s 124ms/step - loss: 0.9823 - accuracy: 0.6624 - val_loss: 1.2631 - val_accuracy: 0.5795
Epoch 176/1000
1/1 [==============================] - 0s 187ms/step - loss: 0.9822 - accuracy: 0.6624 - val_loss: 1.2630 - val_accuracy: 0.5795
Epoch 177/1000
1/1 [==============================] - 0s 168ms/step - loss: 0.9821 - accuracy: 0.6624 - val_loss: 1.2630 - val_accuracy: 0.5795
Epoch 178/1000
1/1 [==============================] - 0s 126ms/step - loss: 0.9821 - accuracy: 0.6624 - val_loss: 1.2629 - val_accuracy:

Epoch 229/1000
1/1 [==============================] - 0s 105ms/step - loss: 0.9779 - accuracy: 0.6634 - val_loss: 1.2594 - val_accuracy: 0.5811
Epoch 230/1000
1/1 [==============================] - 0s 102ms/step - loss: 0.9778 - accuracy: 0.6634 - val_loss: 1.2593 - val_accuracy: 0.5811
Epoch 231/1000
1/1 [==============================] - 0s 210ms/step - loss: 0.9777 - accuracy: 0.6634 - val_loss: 1.2593 - val_accuracy: 0.5811
Epoch 232/1000
1/1 [==============================] - 0s 160ms/step - loss: 0.9776 - accuracy: 0.6634 - val_loss: 1.2592 - val_accuracy: 0.5812
Epoch 233/1000
1/1 [==============================] - 0s 170ms/step - loss: 0.9776 - accuracy: 0.6635 - val_loss: 1.2591 - val_accuracy: 0.5811
Epoch 234/1000
1/1 [==============================] - 0s 127ms/step - loss: 0.9775 - accuracy: 0.6635 - val_loss: 1.2591 - val_accuracy: 0.5811
Epoch 235/1000
1/1 [==============================] - 0s 154ms/step - loss: 0.9774 - accuracy: 0.6635 - val_loss: 1.2590 - val_accuracy:

Epoch 286/1000
1/1 [==============================] - 0s 146ms/step - loss: 0.9734 - accuracy: 0.6649 - val_loss: 1.2557 - val_accuracy: 0.5823
Epoch 287/1000
1/1 [==============================] - 0s 160ms/step - loss: 0.9733 - accuracy: 0.6649 - val_loss: 1.2556 - val_accuracy: 0.5823
Epoch 288/1000
1/1 [==============================] - 0s 121ms/step - loss: 0.9732 - accuracy: 0.6650 - val_loss: 1.2555 - val_accuracy: 0.5823
Epoch 289/1000
1/1 [==============================] - 0s 121ms/step - loss: 0.9732 - accuracy: 0.6650 - val_loss: 1.2555 - val_accuracy: 0.5824
Epoch 290/1000
1/1 [==============================] - 0s 155ms/step - loss: 0.9731 - accuracy: 0.6650 - val_loss: 1.2554 - val_accuracy: 0.5825
Epoch 291/1000
1/1 [==============================] - 0s 125ms/step - loss: 0.9730 - accuracy: 0.6650 - val_loss: 1.2554 - val_accuracy: 0.5825
Epoch 292/1000
1/1 [==============================] - 0s 124ms/step - loss: 0.9729 - accuracy: 0.6650 - val_loss: 1.2553 - val_accuracy:

...

Epoch 742/1000
1/1 [==============================] - 0s 106ms/step - loss: 0.9431 - accuracy: 0.6724 - val_loss: 1.2316 - val_accuracy: 0.5918
Epoch 743/1000
1/1 [==============================] - 0s 137ms/step - loss: 0.9431 - accuracy: 0.6724 - val_loss: 1.2315 - val_accuracy: 0.5918
Epoch 744/1000
1/1 [==============================] - 0s 142ms/step - loss: 0.9430 - accuracy: 0.6724 - val_loss: 1.2315 - val_accuracy: 0.5918
Epoch 745/1000
1/1 [==============================] - 0s 176ms/step - loss: 0.9429 - accuracy: 0.6724 - val_loss: 1.2314 - val_accuracy: 0.5918
Epoch 746/1000
1/1 [==============================] - 0s 155ms/step - loss: 0.9429 - accuracy: 0.6724 - val_loss: 1.2314 - val_accuracy: 0.5918
Epoch 747/1000
1/1 [==============================] - 0s 142ms/step - loss: 0.9428 - accuracy: 0.6724 - val_loss: 1.2313 - val_accuracy: 0.5917
Epoch 748/1000
1/1 [==============================] - 0s 114ms/step - loss: 0.9428 - accuracy: 0.6724 - val_loss: 1.2313 - val_accuracy:

Epoch 799/1000
1/1 [==============================] - 0s 121ms/step - loss: 0.9399 - accuracy: 0.6730 - val_loss: 1.2291 - val_accuracy: 0.5919
Epoch 800/1000
1/1 [==============================] - 0s 157ms/step - loss: 0.9399 - accuracy: 0.6730 - val_loss: 1.2291 - val_accuracy: 0.5919
Epoch 801/1000
1/1 [==============================] - 0s 140ms/step - loss: 0.9398 - accuracy: 0.6731 - val_loss: 1.2290 - val_accuracy: 0.5920
Epoch 802/1000
1/1 [==============================] - 0s 102ms/step - loss: 0.9398 - accuracy: 0.6731 - val_loss: 1.2290 - val_accuracy: 0.5920
Epoch 803/1000
1/1 [==============================] - 0s 104ms/step - loss: 0.9397 - accuracy: 0.6731 - val_loss: 1.2290 - val_accuracy: 0.5919
Epoch 804/1000
1/1 [==============================] - 0s 104ms/step - loss: 0.9396 - accuracy: 0.6731 - val_loss: 1.2289 - val_accuracy: 0.5919
Epoch 805/1000
1/1 [==============================] - 0s 112ms/step - loss: 0.9396 - accuracy: 0.6731 - val_loss: 1.2289 - val_accuracy:

Epoch 856/1000
1/1 [==============================] - 0s 135ms/step - loss: 0.9368 - accuracy: 0.6738 - val_loss: 1.2268 - val_accuracy: 0.5928
Epoch 857/1000
1/1 [==============================] - 0s 105ms/step - loss: 0.9368 - accuracy: 0.6738 - val_loss: 1.2268 - val_accuracy: 0.5928
Epoch 858/1000
1/1 [==============================] - 0s 141ms/step - loss: 0.9367 - accuracy: 0.6738 - val_loss: 1.2267 - val_accuracy: 0.5929
Epoch 859/1000
1/1 [==============================] - 0s 136ms/step - loss: 0.9367 - accuracy: 0.6738 - val_loss: 1.2267 - val_accuracy: 0.5930
Epoch 860/1000
1/1 [==============================] - 0s 139ms/step - loss: 0.9366 - accuracy: 0.6739 - val_loss: 1.2266 - val_accuracy: 0.5931
Epoch 861/1000
1/1 [==============================] - 0s 112ms/step - loss: 0.9366 - accuracy: 0.6739 - val_loss: 1.2266 - val_accuracy: 0.5930
Epoch 862/1000
1/1 [==============================] - 0s 109ms/step - loss: 0.9365 - accuracy: 0.6739 - val_loss: 1.2266 - val_accuracy:

Epoch 913/1000
1/1 [==============================] - 0s 114ms/step - loss: 0.9339 - accuracy: 0.6747 - val_loss: 1.2246 - val_accuracy: 0.5938
Epoch 914/1000
1/1 [==============================] - 0s 154ms/step - loss: 0.9338 - accuracy: 0.6746 - val_loss: 1.2245 - val_accuracy: 0.5937
Epoch 915/1000
1/1 [==============================] - 0s 103ms/step - loss: 0.9338 - accuracy: 0.6747 - val_loss: 1.2245 - val_accuracy: 0.5937
Epoch 916/1000
1/1 [==============================] - 0s 107ms/step - loss: 0.9337 - accuracy: 0.6747 - val_loss: 1.2245 - val_accuracy: 0.5937
Epoch 917/1000
1/1 [==============================] - 0s 135ms/step - loss: 0.9336 - accuracy: 0.6747 - val_loss: 1.2244 - val_accuracy: 0.5937
Epoch 918/1000
1/1 [==============================] - 0s 104ms/step - loss: 0.9336 - accuracy: 0.6747 - val_loss: 1.2244 - val_accuracy: 0.5938
Epoch 919/1000
1/1 [==============================] - 0s 139ms/step - loss: 0.9335 - accuracy: 0.6747 - val_loss: 1.2243 - val_accuracy:

Epoch 970/1000
1/1 [==============================] - 0s 218ms/step - loss: 0.9310 - accuracy: 0.6754 - val_loss: 1.2224 - val_accuracy: 0.5944
Epoch 971/1000
1/1 [==============================] - 0s 195ms/step - loss: 0.9309 - accuracy: 0.6754 - val_loss: 1.2224 - val_accuracy: 0.5944
Epoch 972/1000
1/1 [==============================] - 0s 225ms/step - loss: 0.9309 - accuracy: 0.6754 - val_loss: 1.2224 - val_accuracy: 0.5944
Epoch 973/1000
1/1 [==============================] - 0s 134ms/step - loss: 0.9308 - accuracy: 0.6754 - val_loss: 1.2223 - val_accuracy: 0.5944
Epoch 974/1000
1/1 [==============================] - 0s 143ms/step - loss: 0.9308 - accuracy: 0.6755 - val_loss: 1.2223 - val_accuracy: 0.5944
Epoch 975/1000
1/1 [==============================] - 0s 194ms/step - loss: 0.9307 - accuracy: 0.6755 - val_loss: 1.2223 - val_accuracy: 0.5944
Epoch 976/1000
1/1 [==============================] - 0s 140ms/step - loss: 0.9307 - accuracy: 0.6755 - val_loss: 1.2222 - val_accuracy: